In [ ]:
#This is the Python Flask API code running on pythonanywhere.com
#Different response from the API are generated by using URLs. For example : https://salez.pythonanywhere.com/orders/statistics
#To access other methods of the API use https://salez.pythonanywhere.com/ add the route path. For example: /revenue, /orders 
 


from flask import Flask,jsonify
import pandas as pd
import json






sales_df = pd.read_csv('MajorProjectData.csv')

purchase_df = pd.read_csv('order.csv',encoding = 'unicode_escape',engine='python')

def last_month():
    sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'],format="%d/%m/%Y")
    store_sales = sales_df.copy()
    store_sales['Order Date'] = pd.to_datetime(store_sales['Order Date'],format="%d/%m/%Y")
    store_sales['Order Date'] = store_sales['Order Date'].dt.to_period('m')
    monthly_sales = store_sales.groupby('Order Date').sum().reset_index()
    monthly_sales['Order Date'] = monthly_sales['Order Date'].dt.to_timestamp()
    monthly_sales = monthly_sales[['Order Date','Sales']]
    last_month_sales = monthly_sales['Order Date'].iloc[-1].to_pydatetime()
    second_last_month_sales = monthly_sales['Order Date'].iloc[-2].to_pydatetime()
    return last_month_sales, second_last_month_sales


app = Flask(__name__)


@app.route('/',methods=['GET','POST'])
def Expenses():
    exp = purchase_df['Purchase'].sum()
    data_set = exp
    #json_dump = json.dumps(data_set)
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


@app.route('/revenue',methods=['GET','POST'])
def Revenue():
    store_sales = sales_df.copy()
    store_sales['Order Date'] = pd.to_datetime(store_sales['Order Date'])
    store_sales['Order Date'] = store_sales['Order Date'].dt.to_period('M')
    monthly_sales = store_sales.groupby('Order Date').sum().reset_index()
    monthly_sales['Order Date'] = monthly_sales['Order Date'].dt.to_timestamp()
    monthly_sales = monthly_sales[['Order Date','Sales']]

    last_month_sales =  monthly_sales['Sales'].iloc[-1]
    second_last_month_sales =  monthly_sales['Sales'].iloc[-2]
    sales_difference = ((last_month_sales - second_last_month_sales)/second_last_month_sales)*100

    store_purchase = purchase_df.copy()
    store_purchase['Order Date'] = pd.to_datetime(store_purchase['Order Date'])
    store_purchase['Order Date'] = store_purchase['Order Date'].dt.to_period('M')
    monthly_purchase = store_purchase.groupby('Order Date').sum().reset_index()
    monthly_purchase['Order Date'] = monthly_purchase['Order Date'].dt.to_timestamp()
    monthly_purchase = monthly_purchase[['Order Date','Purchase']]

    last_month_expenses =  monthly_purchase['Purchase'].iloc[-1]
    second_last_month_expenses = 0
    expenses_difference = 0
    if len(monthly_purchase['Purchase'])>1:
        second_last_month_expenses =  monthly_purchase['Purchase'].iloc[-2]
        expenses_difference = ((last_month_expenses - second_last_month_expenses)/second_last_month_expenses)*100

    revenue = last_month_sales - last_month_expenses
    last_month_revenue = second_last_month_sales - second_last_month_expenses
    revenue_difference = ((revenue - last_month_revenue)/last_month_revenue)*100

    data_set = {'last_month_sales':last_month_sales,'last_month_expenses':last_month_expenses,'revenue':revenue,'sales_difference_%':sales_difference,'expenses_difference_%':expenses_difference,'revenue_difference':revenue_difference}
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


@app.route('/salesgraph',methods=['GET','POST'])
def salesgraph():
    sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'],format="%d/%m/%Y")
    store_sales = sales_df.copy()
    store_sales['Order Date'] = pd.to_datetime(store_sales['Order Date'],format="%d/%m/%Y")
    store_sales['Order Date'] = store_sales['Order Date'].dt.to_period('m')
    monthly_sales = store_sales.groupby('Order Date').sum().reset_index()
    monthly_sales['Order Date'] = monthly_sales['Order Date'].dt.to_timestamp()
    monthly_sales = monthly_sales[['Order Date','Sales']]
    last_month_sales = monthly_sales['Order Date'].iloc[-1].to_pydatetime()
    last_month = sales_df[sales_df['Order Date'].dt.month==last_month_sales.month]
    last_month = last_month[last_month['Order Date'].dt.year==last_month_sales.year]
    last_month['Order Date']=[x.to_pydatetime() for x in last_month['Order Date']]
    last_month['Order Date'] = pd.to_datetime(last_month['Order Date']) - pd.to_timedelta(7, unit='d')
    lmdf = last_month.groupby([pd.Grouper(key='Order Date', freq='W')])['Sales'].sum()

    second_last_month_sales = monthly_sales['Order Date'].iloc[-2].to_pydatetime()
    second_last_month = sales_df[sales_df['Order Date'].dt.month==second_last_month_sales.month]
    second_last_month = second_last_month[second_last_month['Order Date'].dt.year==second_last_month_sales.year]
    second_last_month['Order Date']=[x.to_pydatetime() for x in second_last_month['Order Date']]
    second_last_month['Order Date'] = pd.to_datetime(second_last_month['Order Date']) - pd.to_timedelta(7, unit='d')
    lmdf2 = second_last_month.groupby([pd.Grouper(key='Order Date', freq='W')])['Sales'].sum()

    data_set = {'last_month_data':list(lmdf),'second_last_month_data':list(lmdf2),'last_month':str(last_month_sales.date()),'second_last_month':str(second_last_month_sales.date())}
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/productsdonut',methods=['GET','POST'])
def productsdonut():
    df_pr=sales_df[["Product Name","Sales"]]
    df_pr=df_pr.sort_values(by="Sales",ascending=False)
    df_pr=df_pr.groupby("Product Name").sum()
    df_pr=df_pr.reset_index()
    df_pr=df_pr.sort_values(by="Sales",ascending=False,ignore_index=True).head(5)
    data_set = {x:y for x,y in zip(list(df_pr['Product Name']),list(df_pr['Sales']))}
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/salesoverview',methods=['GET','POST'])
def salesoverview():
    pass

@app.route('/categorylist',methods=['GET','POST'])
def categorylist():
    sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'],format="%d/%m/%Y")
    last_month_date,second_last_month_date = last_month()
    last_month_data = sales_df[sales_df['Order Date'].dt.month==last_month_date.month]
    second_last_month_data = sales_df[sales_df['Order Date'].dt.month==second_last_month_date.month]
    top_sales_category_last_month =last_month_data.groupby('Category').sum()['Sales'].reset_index()
    top_categories_last_month = top_sales_category_last_month['Category']
    top_categories_sales_last_month = top_sales_category_last_month['Sales']

    top_sales_category_second_last_month =second_last_month_data.groupby('Category').sum()['Sales'].reset_index()
    top_categories_second_last_month = top_sales_category_second_last_month['Category']
    top_categories_sales_second_last_month = top_sales_category_second_last_month['Sales']



    d1 = dict(zip(top_categories_last_month,top_categories_sales_last_month))
    d2 = dict(zip(top_categories_second_last_month,top_categories_sales_second_last_month))
    sales_cat_diff = {key: ((d1[key] - d2.get(key, 0))/d2.get(key,0))*100 for key in d1}

    data_set = {'last_month_categories':d1, 'second_last_month_categories':d2, '%_difference_between_categories':sales_cat_diff}
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


@app.route('/orders',methods=['GET','POST'])
def orders():
    data_set = { 'order_id':list(purchase_df['Order ID']), 'date':list(purchase_df['Order Date']), 'price':list(purchase_df['Purchase']), 'product_name':list(purchase_df['Product Name'])  }
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/orders/statistics',methods=['GET','POST'])
def orders_statistics():
    top_purchase_category = purchase_df.groupby('Category').sum()['Purchase'].reset_index()
    category = top_purchase_category['Category']
    category_purchase = top_purchase_category['Purchase']
    categorylist = dict(zip(category,category_purchase))

    top_purchase_category = purchase_df.groupby('Segment').sum()['Purchase'].reset_index()
    segment = top_purchase_category['Segment']
    segment_purchase = top_purchase_category['Purchase']
    segmentlist = dict(zip(segment,segment_purchase))

    top_df_pr=purchase_df[["Product Name","Purchase"]]
    top_df_pr=top_df_pr.sort_values(by="Purchase",ascending=False)
    top_df_pr=top_df_pr.groupby("Product Name").sum()
    top_df_pr=top_df_pr.reset_index()
    top_df_pr=top_df_pr.sort_values(by="Purchase",ascending=False,ignore_index=True).head(5)
    top_products_purchase = dict(zip(list(top_df_pr['Product Name']),list(top_df_pr['Purchase'])))

    bottom_df_pr=purchase_df[["Product Name","Purchase"]]
    bottom_df_pr=bottom_df_pr.sort_values(by="Purchase",ascending=True)
    bottom_df_pr=bottom_df_pr.groupby("Product Name").sum()
    bottom_df_pr=bottom_df_pr.reset_index()
    bottom_df_pr=bottom_df_pr.sort_values(by="Purchase",ascending=True,ignore_index=True).head(5)
    bottom_products_purchase = dict(zip(list(bottom_df_pr['Product Name']),list(bottom_df_pr['Purchase'])))

    df_sub=purchase_df[["Sub-Category","Purchase"]]
    df_sub=df_sub.groupby("Sub-Category").sum()
    df_sub=df_sub.reset_index()
    df_sub=df_sub.sort_values(by="Purchase",ascending=True,ignore_index=True)
    subcategorylist = dict(zip(list(df_sub['Sub-Category']),list(df_sub['Purchase'])))

    data_set = {'categorylist':categorylist, 'segmentlist':segmentlist, 'top5products':top_products_purchase, 'bottom5products':bottom_products_purchase, 'subcategorylist':subcategorylist}
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


@app.route('/products',methods=['GET','POST'])
def products():
    #how to send the whole sales dataframe rowwise or columnwise??
    pass



@app.route('/products/statistics',methods=['GET','POST'])
def products_statistics():
    df = sales_df.copy()
    df = df.drop(['Row ID','Customer ID','Order ID','Customer Name'],axis=1)
    df['Order Date'] = pd.to_datetime(df['Order Date'])

    best_products=df[["Product Name","Sales"]]
    best_products=best_products.sort_values(by="Sales",ascending=False)
    best_products=best_products.groupby("Product Name").sum()
    best_products=best_products.reset_index()
    best_products=best_products.sort_values(by="Sales",ascending=False,ignore_index=True).head(10)
    d3 = dict(zip(list(best_products['Product Name']),list(best_products['Sales'])))

    worst_products=df[["Product Name","Sales"]]
    worst_products=worst_products.sort_values(by="Sales",ascending=True)
    worst_products=worst_products.groupby("Product Name").sum()
    worst_products=worst_products.reset_index()
    worst_products=worst_products.sort_values(by="Sales",ascending=True,ignore_index=True).head(10)
    d4 = dict(zip(list(worst_products['Product Name']),list(worst_products['Sales'])))

    dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
    df['weekday'] = df['Order Date'].dt.dayofweek.map(dayOfWeek)
    df_day=df[["weekday",'Sales']]
    df_day=df_day.groupby('weekday').sum()
    df_day=df_day.reset_index()
    d6 = dict(zip(list(df_day['weekday']),list(df_day['Sales'])))

    df["Month"]=df["Order Date"].dt.month_name()
    df_mo=df[["Month","Sales"]]
    df_mo=df_mo.groupby("Month").sum()
    df_mo=df_mo.reset_index()
    d7 = dict(zip(list(df_mo["Month"]),list(df_mo["Sales"])))

    df['Year']=df["Order Date"].dt.year
    df_yr=df[["Year","Sales"]]
    df_yr=df_yr.groupby("Year").sum()
    df_yr=df_yr.reset_index()
    df_yr=df_yr.sort_values(by="Year",ascending=False,ignore_index=True)
    d8 = dict(zip(list(df_yr['Year']),list(df_yr['Sales'])))

    #heatmap doubt table -> react how?

    data_set = { 'top10bestproducts':d3, 'top10worstproducts':d4, 'sales_trend_days':d6, 'sales_trend_month':d7, 'sales_trend_year':d8 }
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

@app.route('/overview',methods=['GET','POST'])
def overview():
    df = sales_df.copy()
    df = df.drop(['Row ID','Customer ID','Order ID','Customer Name'],axis=1)
    df['Order Date'] = pd.to_datetime(df['Order Date'])

    top_sales_category = df.groupby('Category').sum()['Sales'].reset_index()
    d1 = dict(zip(list(top_sales_category['Category']),list(top_sales_category['Sales'])))

    top_sales_segment = df.groupby('Segment').sum()['Sales'].reset_index()
    d2 = dict(zip(list(top_sales_segment['Segment']),list(top_sales_segment['Sales'])))

    df_sub=df[["Sub-Category","Sales"]]
    df_sub=df_sub.groupby("Sub-Category").sum()
    df_sub=df_sub.reset_index()
    df_sub=df_sub.sort_values(by="Sales",ascending=True,ignore_index=True)
    d5 = dict(zip(list(df_sub['Sub-Category']),list(df_sub['Sales'])))

    data_set = { 'categories_wrt_sales':d1, 'segments_wrt_sales':d2, 'subcategories_wrt_sales':d5}
    response = jsonify(data_set)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response


if __name__ == '__main__':
    app.run()
